In [1]:
import modelinference
import prompts
import importlib
import datetime
import torch
from huggingface_hub import login
import utils.modelHelper as mh


from accelerate import Accelerator, notebook_launcher

In [2]:
importlib.reload(modelinference)

<module 'modelinference' from '/project/modelinference.py'>

In [3]:
# Log into Huggingface

with open('pass.txt') as p:
    hf_login = p.read()
    
hf_login = hf_login[hf_login.find('=')+1:hf_login.find('\n')]
login(hf_login, add_to_git_credential=False)

In [4]:
torch.cuda.device_count()

2

In [5]:

run_config = {
    'model_hf_id': 'deepseek-ai/DeepSeek-R1-Distill-Qwen-14B',
    'model_s3_loc': 'deepseek14Q',
    'model_reload': False,
    'model_quant': None,
    'system_prompt': prompts.SYSTEM_PROMPTS['CoT'],
    'multi-gpu':True,
    'dataset': 'data_annual_pit_spx',
    'data_location': 'data_annual_pit_spx.json'
}

run_config = {
    'model_hf_id': 'Qwen/Qwen2.5-7B-Instruct',
    'model_s3_loc': 'qwen',
    'model_reload': False,
    'model_quant': None,
    'system_prompt': prompts.SYSTEM_PROMPTS['BASE'],
    'multi-gpu':True,
    'dataset': 'data_quarterly_pit_indu',
    'data_location': 'data_quarterly_pit_indu.json'
}

run_config = {
    'model_hf_id': 'meta-llama/Llama-3.2-3B-Instruct',
    'model_s3_loc': 'llama',
    'model_reload': False,
    'model_quant': None,
    'system_prompt': prompts.SYSTEM_PROMPTS['BASE'],
    'multi-gpu':True,
    'dataset': 'data_quarterly_pit_indu',
    'data_location': 'data_quarterly_pit_indu.json'
}




run_name = f"{run_config['model_s3_loc']} - {run_config['dataset']}"

In [9]:
ir = modelinference.InferenceRun(run_name, run_config)

In [10]:
notebook_launcher(ir.run_multi_gpu, num_processes=torch.cuda.device_count())

Launching training on 2 GPUs.
llama
llama


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

waiting...
Loading Data


  0%|          | 0/8 [00:00<?, ?it/s]

...

running model...
running model...
running model...


100%|██████████| 8/8 [01:30<00:00, 10.54s/it]

running model...
Finished run


100%|██████████| 8/8 [02:52<00:00, 21.52s/it]
[2025-02-22 19:12:04,281] torch.distributed.elastic.multiprocessing.api: [ERROR] failed (exitcode: 1) local_rank: 0 (pid: 5341) of fn: InferenceRun.run_multi_gpu (start_method: fork)


ChildFailedError: 
============================================================
run_multi_gpu FAILED
------------------------------------------------------------
Failures:
  <NO_OTHER_FAILURES>
------------------------------------------------------------
Root Cause (first observed failure):
[0]:
  time      : 2025-02-22_19:12:03
  host      : 8274481-prj47495909d3fe47e88cd6d023148eb07d-ed6608454c-pod
  rank      : 0 (local_rank: 0)
  exitcode  : 1 (pid: 5341)
  error_file: /tmp/torchelastic_sy2ocdf0/none_mrbsq0qk/attempt_0/0/error.json
  traceback : Traceback (most recent call last):
    File "/opt/kernel/lib/python3.11/site-packages/torch/distributed/elastic/multiprocessing/errors/__init__.py", line 346, in wrapper
      return f(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^
    File "/project/modelinference.py", line 222, in run_multi_gpu
      self.save_run(results_gathered)
    File "/project/modelinference.py", line 109, in save_run
      self.logger.log(results, f"results - {s

In [35]:
ir = modelinference.InferenceRun(run_name, run_config)

In [26]:
p1 = ir.create_all_prompts()

In [27]:
len(p1)

896

In [22]:
model = ir.load_model_from_storage(ir.model_s3_loc)

llama


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
from transformers import AutoTokenizer

In [24]:
tokenizer = AutoTokenizer.from_pretrained(ir.model_hf_id)

In [28]:
output_result = ir.run_model(p1[0]['prompt'],tokenizer,model)

running model...


In [18]:
output_result = "Based on the provided income statement and balance sheet data, I have computed the following financial ratios:\n\n1. Current Ratio: \n   Current Ratio = Current Assets / Current Liabilities\n   = 9.222000e+09 / 9.222000e+09 = 1\n\n2. Debt-to-Equity Ratio: \n   Debt-to-Equity Ratio = Total Liabilities / Total Equity\n   = 3.453300e+10 / 1.012600e+10 = 3.39\n\n3. Return on Equity (ROE): \n   ROE = Net Income / Total Equity\n   = 1.012600e+10 / 1.076400e+10 = 0.943\n\n4. Return on Assets (ROA): \n   ROA = Net Income / Total Assets\n   = 1.012600e+10 / 4.465900e+10 = 0.228\n\n5. Interest Coverage Ratio: \n   Interest Coverage Ratio = Earnings Before Interest and Taxes (EBIT) / Interest Expense\n   EBIT = Net Income + Depreciation + Amortization\n   = 1.012600e+10 + 1.679100e+10 + 1.008300e+10 = 3.700100e+10\n   Interest Expense = 1.240000e+08 + 1.110000e+08 + 1.800000e+07 = 1.960000e+08\n   Interest Coverage Ratio = 3.700100e+10 / 1.960000e+08 = 1.89\n\nBased on these ratios, I am inclined to recommend a BUY decision with a confidence score of 70%. The company has a relatively low debt-to-equity ratio and a high return on equity, indicating a strong financial position. The interest coverage ratio is also high, suggesting that the company can meet its interest payments. However, the current ratio is 1, which may indicate liquidity concerns. Further analysis of the company's cash flow and debt repayment plans is necessary to confirm this decision."

In [23]:
import json

In [20]:
def format_json(llm_output):
    # remove all the broken lines
    form = llm_output.replace('\n','')
    # Find the start and end of the JSON input
    try:
        soj = form.find('```json')
        eoj = form.find('}```')

        if eoj == -1:
            eoj = len(llm_output)
        # Pull out the additional context
        additional = form[:soj]
        additional += form[eoj + 4:]
        json_obj = json.load(form[soj + 7:eoj + 1])
        json_obj['AdditionalContext'] = additional
        return json_obj
    except:
        return llm_output

In [21]:
format_json(output_result)

"Based on the provided income statement and balance sheet data, I have computed the following financial ratios:\n\n1. Current Ratio: \n   Current Ratio = Current Assets / Current Liabilities\n   = 9.222000e+09 / 9.222000e+09 = 1\n\n2. Debt-to-Equity Ratio: \n   Debt-to-Equity Ratio = Total Liabilities / Total Equity\n   = 3.453300e+10 / 1.012600e+10 = 3.39\n\n3. Return on Equity (ROE): \n   ROE = Net Income / Total Equity\n   = 1.012600e+10 / 1.076400e+10 = 0.943\n\n4. Return on Assets (ROA): \n   ROA = Net Income / Total Assets\n   = 1.012600e+10 / 4.465900e+10 = 0.228\n\n5. Interest Coverage Ratio: \n   Interest Coverage Ratio = Earnings Before Interest and Taxes (EBIT) / Interest Expense\n   EBIT = Net Income + Depreciation + Amortization\n   = 1.012600e+10 + 1.679100e+10 + 1.008300e+10 = 3.700100e+10\n   Interest Expense = 1.240000e+08 + 1.110000e+08 + 1.800000e+07 = 1.960000e+08\n   Interest Coverage Ratio = 3.700100e+10 / 1.960000e+08 = 1.89\n\nBased on these ratios, I am incline

In [33]:
form = output_result.replace('\n','')

In [12]:
s = "hello"

In [14]:
s.find('z')

-1

In [29]:
helper = mh.ModelHelper('tmp/fs')

In [30]:
helper.clear_folder('llama')